# Preparing the necessary libraries and results

In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd

# import the datetime library to work with dates
import datetime
from datetime import datetime, date, timedelta

# Set some pandas library options that 
# configure output
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)
# disable exponential notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Let's prepare data according to the mt4 and mt5 schemes

## mt4 preparing data for analysis

In [2]:
# write a CSV file to a DataFrame object
mt4_trades = pd.read_csv(r"interview_1_exmpl_mt4_trades.csv", sep = ',')
mt4_marked_trades = pd.read_csv(r"interview_1_exmpl_mt4_marked_trades.csv", sep = ',')

The mt4.trades table contains information about client transactions; it contains the following columns:

    ticket     – primary key, deal identifier
    login      – login of the client who made the transaction
    open_time  – time of opening a trade
    close_time – time of closing the deal. If the deal is not closed there is a zero time stamp
    symbol     – symbol, instrument by which the transaction was made
    cmd        – 0 if the deal is to buy, and 1 if the deal is to sell
Thus, each transaction corresponds to 1 record.

The mt4.marked_trades table contains information about marked trades. 
Some of these transactions will need to be excluded from consideration (see below)

    ticket – primary key, deal identifier
    type – a bit mask that indicates the reason for the marking.
If the first bit of the type mask (numbering from 0) is 1, then such a transaction does not need to be taken into account

In [3]:
mt4_trades

        Unnamed: 0     ticket      login            open_time           close_time  symbol  cmd  volume
0                0  372752301  290627505  2022-02-03 17:33:01  2022-02-03 17:39:00  EURUSD    0       3
1                1  372752584  290627505  2022-02-03 17:34:02  2022-02-03 17:39:00  EURUSD    0       4
2                2  372752581  290627505  2022-02-03 17:34:02  2022-02-03 17:39:04  EURUSD    0       3
3                3  372752582  290627505  2022-02-03 17:34:02  2022-02-03 17:39:04  EURUSD    0       3
4                4  372752586  290627505  2022-02-03 17:34:02  2022-02-03 17:39:04  EURUSD    0       3
...            ...        ...        ...                  ...                  ...     ...  ...     ...
265634      265634  372727735  290652685  2022-02-03 16:01:00  2022-02-11 21:02:11  EURUSD    1       5
265635      265635  373022961  290652685  2022-02-07 17:22:32  2022-02-08 09:21:30  XAUUSD    1      33
265636      265636  373034944  290652685  2022-02-07 18:33:19  2

In [4]:
mt4_marked_trades

       Unnamed: 0     ticket  type
0               0  372901954     2
1               1  372743999     1
2               2  373053139     2
3               3  372612581     2
4               4  372640437     2
...           ...        ...   ...
15113       15113  372522040    32
15114       15114  372965069     2
15115       15115  372934267    16
15116       15116  372465264    16
15117       15117  372421070    16

[15118 rows x 3 columns]

In [5]:
# Add a new column of excluded transactions isExcluded binary
# according to the condition 
# (If the first bit of the type mask (numbering from 0) is equal to 1, 
# then such a transaction does not need to be taken into account)

mt4_marked_trades['isExcluded'] = mt4_marked_trades.type &2 == 2
mt4_marked_trades

       Unnamed: 0     ticket  type  isExcluded
0               0  372901954     2        True
1               1  372743999     1       False
2               2  373053139     2        True
3               3  372612581     2        True
4               4  372640437     2        True
...           ...        ...   ...         ...
15113       15113  372522040    32       False
15114       15114  372965069     2        True
15115       15115  372934267    16       False
15116       15116  372465264    16       False
15117       15117  372421070    16       False

[15118 rows x 4 columns]

In [6]:
# Create a new data slice generalized with keys, and those that are not in the marked ones are marked as not to be excluded
mt4_data = mt4_trades.merge(mt4_marked_trades, on='ticket', how='left').fillna(False)
# Remove excluded ones
mt4_data = mt4_data[(mt4_data.isExcluded) == False]
# Remove the extra columns created after join
mt4_data.drop(['Unnamed: 0_x', 'Unnamed: 0_y','type','isExcluded'], axis=1, inplace=True) 
# Correct date and time formats
mt4_data['open_time'] = pd.to_datetime(mt4_data['open_time'])
mt4_data['close_time'] = pd.to_datetime(mt4_data['close_time'])
mt4_data

           ticket      login           open_time          close_time  symbol  cmd  volume
0       372752301  290627505 2022-02-03 17:33:01 2022-02-03 17:39:00  EURUSD    0       3
1       372752584  290627505 2022-02-03 17:34:02 2022-02-03 17:39:00  EURUSD    0       4
2       372752581  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  EURUSD    0       3
3       372752582  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  EURUSD    0       3
4       372752586  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  EURUSD    0       3
...           ...        ...                 ...                 ...     ...  ...     ...
265634  372727735  290652685 2022-02-03 16:01:00 2022-02-11 21:02:11  EURUSD    1       5
265635  373022961  290652685 2022-02-07 17:22:32 2022-02-08 09:21:30  XAUUSD    1      33
265636  373034944  290652685 2022-02-07 18:33:19 2022-02-08 09:21:30  XAUUSD    1      59
265637  372737533  290652685 2022-02-03 16:36:01 2022-02-11 20:46:19  EURJPY    1       5
265638  37

## mt5 preparing data for analysis

In [7]:
# write a CSV file to a DataFrame object
mt5_deals = pd.read_csv(r"C:\Users\fekuz\OneDrive\Документы\Gleb\Work\Serbia\Data-Driven Lab\mt5_deals.csv", sep = ',')
mt5_marked_trades = pd.read_csv(r"C:\Users\fekuz\OneDrive\Документы\Gleb\Work\Serbia\Data-Driven Lab\mt5_marked_trades.csv", sep = ',')

The mt5.deals table contains information about customer transactions. It has columns:

    deal – primary key, unique event identifier
    positionid – order (deal) number, unique deal identifier
    login – login of the client who made the transaction
    time – event time
    symbol – symbol, instrument on which the transaction was made
    action – 0 if the deal is to buy, and 1 if the deal is to sell
    entry – 0 if this event is the opening of a deal, 1 – otherwise.
A deal can be closed by several events
Thus, each transaction can have many records.

The mt5.marked_trades table contains information about marked trades. 
Some of these transactions will need to be excluded from consideration (see below)

    positionid – primary key, deal identifier
    type – a bit mask that indicates the reason for the marking.
If the first bit of the type mask (numbering from 0) is 1, then such a transaction does not need to be taken into account

In [8]:
# Convert float to int
mt5_deals['deal'] = mt5_deals['deal'].astype(np.int64)
mt5_deals['positionid'] = mt5_deals['positionid'].astype(np.int64)
mt5_deals

        Unnamed: 0        deal  positionid     login                 time symbol  action  entry
0                0  2411908473  2505364436  22837778  2022-02-04 17:34:08   US30       0      0
1                1  2411908840  2505364436  22837778  2022-02-04 17:34:51   US30       1      1
2                2  2411908881  2505364838  22837778  2022-02-04 17:34:57   US30       0      0
3                3  2411909170  2505365126  22837778  2022-02-04 17:35:37   US30       0      0
4                4  2411909462  2505365431  22837778  2022-02-04 17:36:15   US30       0      0
...            ...         ...         ...       ...                  ...    ...     ...    ...
454221      454221  2411905915  2505361768  22837778  2022-02-04 17:29:04   US30       0      0
454222      454222  2411906083  2505361935  22837778  2022-02-04 17:29:22   US30       0      0
454223      454223  2411906235  2505362084  22837778  2022-02-04 17:29:39   US30       0      0
454224      454224  2411906396  25053622

In [9]:
# Convert float to int
mt5_marked_trades['positionid'] = mt5_marked_trades['positionid'].astype(np.int64)
mt5_marked_trades

       Unnamed: 0  positionid  type
0               0  2503897459    32
1               1  2503974569     2
2               2  2503930677    16
3               3  2503488064    32
4               4  2505655740    16
...           ...         ...   ...
19916       19916  2504153085    32
19917       19917  2505439027    16
19918       19918  2503973632    32
19919       19919  2503887125     8
19920       19920  2503685491     8

[19921 rows x 3 columns]

In [10]:
# Add a new column of excluded transactions isExcluded binary
# according to the condition 
# (If the first bit of the type mask (numbering from 0) is equal to 1, 
# then such a transaction does not need to be taken into account)
mt5_marked_trades['isExcluded'] = mt5_marked_trades.type &2 == 2
mt5_marked_trades

       Unnamed: 0  positionid  type  isExcluded
0               0  2503897459    32       False
1               1  2503974569     2        True
2               2  2503930677    16       False
3               3  2503488064    32       False
4               4  2505655740    16       False
...           ...         ...   ...         ...
19916       19916  2504153085    32       False
19917       19917  2505439027    16       False
19918       19918  2503973632    32       False
19919       19919  2503887125     8       False
19920       19920  2503685491     8       False

[19921 rows x 4 columns]

In [11]:
# Create a new data slice generalized with keys, 
# and those that are not in the marked ones are marked as not to be excluded
mt5_data = mt5_deals.merge(mt5_marked_trades, on='positionid', how='left').fillna(False)
# Remove excluded ones
mt5_data = mt5_data[(mt5_data.isExcluded) == False]
# Remove the extra columns created after join
mt5_data.drop(['Unnamed: 0_x', 'Unnamed: 0_y','type','isExcluded'], axis=1, inplace=True) 
# Correct date and time formats
mt5_data['time'] = pd.to_datetime(mt5_data['time'])
mt5_data

              deal  positionid     login                time symbol  action  entry
3       2411909170  2505365126  22837778 2022-02-04 17:35:37   US30       0      0
4       2411909462  2505365431  22837778 2022-02-04 17:36:15   US30       0      0
5       2411910104  2505366058  22837778 2022-02-04 17:37:50   US30       0      0
6       2411910149  2505366058  22837778 2022-02-04 17:38:00   US30       1      1
7       2411910170  2505366149  22837778 2022-02-04 17:38:04   US30       0      0
...            ...         ...       ...                 ...    ...     ...    ...
454221  2411905915  2505361768  22837778 2022-02-04 17:29:04   US30       0      0
454222  2411906083  2505361935  22837778 2022-02-04 17:29:22   US30       0      0
454223  2411906235  2505362084  22837778 2022-02-04 17:29:39   US30       0      0
454224  2411906396  2505362269  22837778 2022-02-04 17:29:54   US30       0      0
454225  2411907350  2505362269  22837778 2022-02-04 17:31:09   US30       1      1

[43

# Problem 2.1 Find: For each user, the number of such transactions for which less than one minute passed from the moment of opening to the moment of complete closure

## mt4

In [12]:
# Let's add a new column with the value of the deal interval
mt4_data['Interval'] = mt4_data.close_time - mt4_data.open_time
# Add a column Task_2_1, 
# whose value is the frequency of login categories with transactions less than 60 seconds in time
mt4_data['Task_2_1'] = mt4_data['login'].map(mt4_data[(mt4_data.Interval) < timedelta(seconds=60)].groupby('login').count().Interval)
# Fill the NaN data with the value 0
mt4_data['Task_2_1'].fillna(0, inplace=True)
# Convert float to int
mt4_data['Task_2_1'] = mt4_data['Task_2_1'].astype(int)
mt4_data

           ticket      login           open_time          close_time  ... cmd  volume        Interval Task_2_1
0       372752301  290627505 2022-02-03 17:33:01 2022-02-03 17:39:00  ...   0       3 0 days 00:05:59      173
1       372752584  290627505 2022-02-03 17:34:02 2022-02-03 17:39:00  ...   0       4 0 days 00:04:58      173
2       372752581  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  ...   0       3 0 days 00:05:02      173
3       372752582  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  ...   0       3 0 days 00:05:02      173
4       372752586  290627505 2022-02-03 17:34:02 2022-02-03 17:39:04  ...   0       3 0 days 00:05:02      173
...           ...        ...                 ...                 ...  ...  ..     ...             ...      ...
265634  372727735  290652685 2022-02-03 16:01:00 2022-02-11 21:02:11  ...   1       5 8 days 05:01:11       25
265635  373022961  290652685 2022-02-07 17:22:32 2022-02-08 09:21:30  ...   1      33 0 days 15:58:58       25
2

## mt5

In [13]:
# Add a trade opening date column
mt5_data['minOpenTime'] = mt5_data['positionid'].map(mt5_data[(mt5_data.entry) == 0].groupby('positionid').time.min())
# Add a column for the date of complete closure of the transaction
mt5_data['maxCloseTime'] = mt5_data['positionid'].map(mt5_data[(mt5_data.entry) == 1].groupby('positionid').time.max())
# Add a new column with the trade interval value
mt5_data['Interval'] = mt5_data.maxCloseTime - mt5_data.minOpenTime
# Add a column Task_2_1, whose value is the frequency 
# of login categories with transactions less than 60 seconds in time
mt5_data['Task_2_1'] = mt5_data['login'].map(mt5_data[(mt5_data.Interval) < timedelta(seconds=60)].groupby('login').agg(cntUniqPos =('positionid', 'nunique')).cntUniqPos)
# Fill the NaN data with the value 0
mt5_data['Task_2_1'].fillna(0, inplace=True)
# Convert float to int
mt5_data['Task_2_1'] = mt5_data['Task_2_1'].astype(int)
mt5_data

              deal  positionid     login                time  ...         minOpenTime        maxCloseTime  \
3       2411909170  2505365126  22837778 2022-02-04 17:35:37  ... 2022-02-04 17:35:37 2022-02-04 18:27:01   
4       2411909462  2505365431  22837778 2022-02-04 17:36:15  ... 2022-02-04 17:36:15 2022-02-04 18:25:49   
5       2411910104  2505366058  22837778 2022-02-04 17:37:50  ... 2022-02-04 17:37:50 2022-02-04 17:38:00   
6       2411910149  2505366058  22837778 2022-02-04 17:38:00  ... 2022-02-04 17:37:50 2022-02-04 17:38:00   
7       2411910170  2505366149  22837778 2022-02-04 17:38:04  ... 2022-02-04 17:38:04 2022-02-04 17:40:12   
...            ...         ...       ...                 ...  ...                 ...                 ...   
454221  2411905915  2505361768  22837778 2022-02-04 17:29:04  ... 2022-02-04 17:29:04 2022-02-04 18:46:04   
454222  2411906083  2505361935  22837778 2022-02-04 17:29:22  ... 2022-02-04 17:29:22 2022-02-04 18:35:21   
454223  2411906235 